In [1]:
import sqlite3
from datetime import datetime
from pdb import set_trace

In [24]:
class DBHandler:
    '''
    Handles all the querying from our database
    '''
    
    def __init__(
        self,
        conn: sqlite3.Connection
    ) -> None:
        
        self.conn = conn
        self.cursor = self.conn.cursor()
        
        self.create_table()
        
    
    def create_table(
        self
    ):
        '''
        Creates a table if does not already exist
        '''
        
        query = '''
        CREATE TABLE IF NOT EXISTS readings(
            id INT PRIMARY KEY,
            temperature FLOAT,
            oxygen INT,
            time TIMESTAMP
        )
        '''
        with self.conn:
            self.cursor.execute(query)
            
    
    def get_latest_id(
        self
    ) -> int:
        query = '''
        SELECT MAX(id) FROM readings
        '''
        self.cursor.execute(query)
        max_id = self.cursor.fetchone()[0]
        return max_id + 1 if max_id else 1
            
            
    def set_temp_and_o2(
        self,
        temp: float,
        o2: int
    ) -> None:
        '''
        Inserts a new row with the given readings
        '''
        
        curr_id = self.get_latest_id()
        curr_time = datetime.now()
        
        query = f'''
        INSERT INTO readings VALUES(
            {curr_id},
            {temp},
            {o2},
            '{curr_time}'
        )
        '''
        
        with self.conn:
            self.cursor.execute(query)
            
    def get_n_latest_readings(
        self,
        n_readings: int
    ) -> list:
        '''
        Retrieves n latest readings
        '''
        
        n_readings = min(n_readings, self.get_latest_id())
        
        query = f'''
        SELECT temperature, oxygen FROM readings
        ORDER BY id DESC
        LIMIT {n_readings}
        '''
        
        self.cursor.execute(query)
        return self.cursor.fetchall()

In [ ]:
if __name__ == '__main__':
    conn = sqlite3.connect('readings.db', detect_types=sqlite3.PARSE_DECLTYPES)
    obj = DBHandler()
    
    obj.get_temp_and_o2(3)
    
    handler = DBHandler(conn)
    handler.set_temp_and_o2(temp = 98, o2 = 96)
    handler.set_temp_and_o2(temp = 96, o2 = 94)

    handler.get_n_latest_readings(4)